<a href="https://colab.research.google.com/github/nematatu/Colab-Python/blob/master/Hello.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install --upgrade tensorflow

In [4]:
pip install keras

In [5]:
import tensorflow as tf

In [6]:
#データセットをmnistに入れる
mnist=tf.keras.datasets.mnist
#mnist.load_data()とすることで、トレーニングデータとテストデータが得られる
#配列とか、カンマで分けられている値をそれぞれに代入できる
(x_train,y_train),(x_test,y_test)=mnist.load_data()
#画像データが255なので、0-1に変換
x_train,x_test=x_train/255.0,x_test/255.0

11490434/11490434 [==============================] - 1s 0us/step


In [7]:
#Sequentialモデルを作る
model=tf.keras.models.Sequential([
    #28x28でフラット化する
    tf.keras.layers.Flatten(input_shape=(28,28)),
    #Dense(密度)層で、活性化関数reluを使って、128ノードで分析
    tf.keras.layers.Dense(128,activation='relu'),
    #全体の内20%のみを使うことで、過学習を防ぐ
    tf.keras.layers.Dropout(0.2),
    #最終的に10このデータを返す
    tf.keras.layers.Dense(10)
])

In [8]:
#↑で作ったモデルに、x_trainの一番目の要素を適応させて、numpy()配列にする
#Sequentialに__call__という、クラスを関数のように引数を取って実行できるメソッドが組み込まれている
predictions=model(x_train[:1]).numpy()
predictions

array([[ 0.11737003,  0.07767689, -0.19360083,  0.47782302, -0.3672945 ,
        -0.0595631 , -0.7830403 ,  0.34175572, -0.23714766,  0.5641862 ]],
      dtype=float32)

In [9]:
#tfのnnのsoftmax関数を呼び出して数値を確率分布にする
tf.nn.softmax(predictions).numpy()

array([[0.10521516, 0.10112064, 0.07709485, 0.15087631, 0.06480242,
        0.08815304, 0.04275974, 0.13168243, 0.07380966, 0.1644857 ]],
      dtype=float32)

In [10]:
#損失計算を提供するモデルを構築する
#from_logits=Trueとすると、logits(ソフトマックス関数を適用する前の状態=確率分布に指定ない状態)で出力する
loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [11]:
#__call__メソッドが組み込まれているので関数のように引数を取って実行できる
#loss_fn(調べるラベル,モデルの予想結果)
#とすることで、モデルの予想結果(理想値)からみて損失計算ができる
loss_fn(y_train[:1],predictions).numpy()

2.428681

In [13]:
#モデルのコンパイルとは、モデルが訓練可能な状態になるための設定を行うこと
#optimizerは最適化アルゴリズム。いろんなのある。
#lossは損失関数
#metricsは評価指数。いろんなのある。
model.compile(optimizer='adam',loss=loss_fn,metrics=['accuracy'])

In [14]:
#model.fitでkerasモデルをトレーニングする
#epochsはトレーニング回数
model.fit(x_train,y_train,epochs=5)

Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2943 - accuracy: 0.9142
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1433 - accuracy: 0.9575
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1080 - accuracy: 0.9671
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0893 - accuracy: 0.9728
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0764 - accuracy: 0.9760


In [15]:
#トレーニング済みのモデルを使用して、テストデータを評価する
#verboseは、詳細ログを表示するための設定。verbose=2は最も詳細なログを出す。
model.evaluate(x_test,y_test,verbose=2)

313/313 - 1s - loss: 0.0747 - accuracy: 0.9765 - 581ms/epoch - 2ms/step


[0.07468167692422867, 0.9764999747276306]

In [17]:
#学習させたモデルに確率を返させる
probability_model=tf.keras.Sequential([model,tf.keras.layers.Softmax()
])

In [18]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.4996285e-07, 1.4709005e-08, 1.4444710e-05, 1.1764787e-03,
        2.1692380e-10, 6.3592131e-07, 8.2269322e-12, 9.9876416e-01,
        5.1301108e-06, 3.9010651e-05],
       [1.5643604e-08, 1.4724216e-03, 9.9851900e-01, 4.7247527e-06,
        2.4671641e-17, 3.6947773e-07, 5.1227598e-08, 1.1000116e-13,
        3.4455584e-06, 4.9432546e-12],
       [3.1873455e-07, 9.9944597e-01, 9.7487398e-05, 1.3002038e-05,
        1.2318808e-05, 4.7472136e-06, 2.3743540e-05, 2.3853855e-04,
        1.6030138e-04, 3.4626414e-06],
       [9.9545896e-01, 5.1820365e-08, 2.7308561e-04, 3.1433927e-05,
        4.4286554e-07, 2.3965678e-05, 4.0099416e-03, 2.7607013e-05,
        1.9655724e-06, 1.7242375e-04],
       [8.0956024e-06, 1.8213028e-08, 1.6578590e-05, 4.7656926e-07,
        9.9060506e-01, 1.4169773e-06, 1.4621279e-05, 1.7122649e-04,
        5.6047588e-07, 9.1819791e-03]], dtype=float32)>